# Develop  plots
author: [Mathieu Renzo](mrenzo@flatironinstitute.org)

In [ ]:
import sys
# go fetch libraries
sys.path.append("../figures/utils/")
from MESAreader import getSrcCol, read_MESA_header, Rsun_cm
sys.path.append("/mnt/home/mrenzo/codes/python_stuff/plotFunc/")
from plot_defaults import *
import glob
from scipy.interpolate import interp1d
import numpy as np
from lib_plot_bin import plot_BE_r, plot_BE_m,\
    interpolate_BE_env, get_ax_from_pfile,\
    plot_surface_abundances, get_lambda_profile, \
    plot_lambda_mass, plot_lambda_at_one_radius, plot_lambda_r, get_ratio_BE
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2

In [ ]:
set_plot_defaults_from_matplotlibrc("../figures/")

In [ ]:
root = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/"
root_accretors = root + "binaries/Z_0.0019/"

b1 = (
    root_accretors
    + "/m1_18.0000_m2_15.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/LOGS2/"
)
b2 = (
    root_accretors
    + "/m1_20.0000_m2_17.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/LOGS2/"
)
b3 = (
    root_accretors
    + "/m1_38.0000_m2_30.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/LOGS2/"
)
nonrot36 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/single_stars/Z_0.0019/36_rot0.0/"
nonrot20 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/single_stars/Z_0.0019/20_rot0.0/"
nonrot18 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/single_stars/Z_0.0019/18_rot0.0/"

root_simplified = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/same_core/"
root_grid15 = root_simplified+"grid15/"
root_grid17 = root_simplified+"grid17/"
root_grid18 = root_simplified+"grid18/"
root_grid20 = root_simplified+"grid20/"
root_grid30 = root_simplified+"grid30/"
root_grid34 = root_simplified+"grid34/"
root_grid35 = root_simplified+"grid35/"
root_grid36 = root_simplified+"grid36/"
grid_folders15 = sorted(glob.glob(root_grid15+"/*.*/"))
grid_folders17 = sorted(glob.glob(root_grid17+"/*.*/"))
grid_folders18 = sorted(glob.glob(root_grid18+"/*.*/"))
grid_folders20 = sorted(glob.glob(root_grid20+"/*.*/"))
grid_folders30 = sorted(glob.glob(root_grid30+"/*.*/"))
grid_folders34 = sorted(glob.glob(root_grid34+"/*.*/"))
grid_folders35 = sorted(glob.glob(root_grid35+"/*.*/"))
grid_folders36 = sorted(glob.glob(root_grid36+"/*.*/"))
init_model15 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/15_rot0_to_TAMS/LOGS/TAMS.data"
init_model17 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/17_rot0_to_TAMS/LOGS/TAMS.data"
init_model18 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/18_rot0_to_TAMS/LOGS/TAMS.data"
init_model20 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/20_rot0_to_TAMS/LOGS/TAMS.data"
init_model30 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/30_rot0_to_TAMS/LOGS/TAMS.data"
init_model34 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/34_rot0_to_TAMS/LOGS/TAMS.data"
init_model35 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/35_rot0_to_TAMS/LOGS/TAMS.data"
init_model36 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/engineered_stars/36_rot0_to_TAMS/LOGS/TAMS.data"


In [ ]:
# on BE ratio
fig = plt.figure()
gs = gridspec.GridSpec(100, 120)
gs.update(wspace=0,hspace=0)# top=1.1)
ax = fig.add_subplot(gs[:,:])

ax.set_ylabel(r"$BE(\mathrm{accretor})/BE(\mathrm{single})$")
ax.set_xlabel(r"$\log_{10}(r/\mathrm{[cm]})$")
ax.axhline(1, 0, 1, lw=2, ls="--", c="#808080")
ax.text(
    0.04,
    0.9,
    "accretor more bound",
    fontsize=20,
    transform=ax.transAxes,
    va="bottom",
    ha="left",
    zorder=0,
)
ax.text(
    0.04,
    0.05,
    "accretor less bound",
    fontsize=20,
    transform=ax.transAxes,
    va="bottom",
    ha="left",
    zorder=0,
)


f_acc = b2+'/'
f_sin = nonrot20+'LOGS/'

for string in ["100", "200", "300", "500"]:
    pfile_acc = f_acc+'/'+string+'Rsun.data'
    pfile_sin = f_sin+'/'+string+'Rsun.data'

    ratio = get_ratio_BE(pfile_sin, pfile_acc, alpha_th=1.0)
    src, col = getSrcCol(pfile_sin)
    logr = np.log10(src[:, col.index("radius")]*Rsun_cm)
    ax.plot(logr, ratio, label=string+"$R_\odot$")
ax.set_title(r"$17M_\odot \rightarrow 20M_\odot$", size=30)
ax.set_xlim(8.2, 14)
ax.legend(fontsize=20, handlelength=0.5, loc="upper left",
          bbox_to_anchor=(0.01,0.85),
          ncol=2, columnspacing=0.5, frameon=True)
plt.savefig('/mnt/home/mrenzo/TMP/ratio_binding_energy_profiles.pdf')

## lambda profiles

In [ ]:
# test
# ----------------------
# what to plot below
grid_folders = grid_folders20
init_model = init_model20
accretor = b2
nonrot = nonrot20
# --------------------------

fig = plt.figure()
gs = gridspec.GridSpec(100, 120)
gs.update(wspace=0,hspace=0)# top=1.1)
ax = fig.add_subplot(gs[:,:])
plot_lambda_at_one_radius(ax, "200Rsun.data", grid_folders, accretor, nonrot, plot_func = plot_lambda_mass, legend=False)
ax.set_xlabel(r"m [$M_\odot$]")
ax.set_ylabel(r"$\lambda = (GM(M-m)/R)/\mathrm{BE(m)}}$")

In [ ]:
def grid_lambdas(b1, b2, b3, nonrot1, nonrot2, nonrot3, grid_folders1, grid_folders2, grid_folders3, plot_func=plot_lambda_mass, legend=False):
    fig = plt.figure(figsize=(16, 24))
    gs = gridspec.GridSpec(120, 150)
    # # 15Msun accretor
    ax1 = fig.add_subplot(gs[:20, :50])
    ax2 = fig.add_subplot(gs[20:40, :50])
    ax3 = fig.add_subplot(gs[40:60, :50])
    ax4 = fig.add_subplot(gs[60:80, :50])
    ax5 = fig.add_subplot(gs[80:100, :50])
    axes = [ax1, ax2, ax3, ax4, ax5]
    for ax in axes:
        ax.set_xlim(0, 18.1)
        if ax != axes[-1]:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel(r"$m [M_\odot]$")
    ax1.set_ylim(-0.1,1.1)
    ax2.set_ylim(-0.1,1.1)
    ax3.set_ylim(-0.1,1.3)
    ax4.set_ylim(-0.1, 1.7)
    ax5.set_ylim(-0.1, 2.1)
    plot_lambda_at_one_radius(ax1, "100Rsun.data", grid_folders=grid_folders1 ,accretor=b1, nonrot=nonrot1, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(ax2, "200Rsun.data", grid_folders=grid_folders1 ,accretor=b1, nonrot=nonrot1, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(ax3, "300Rsun.data", grid_folders=grid_folders1 ,accretor=b1, nonrot=nonrot1, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(ax4, "500Rsun.data", grid_folders=grid_folders1 ,accretor=b1, nonrot=nonrot1, plot_func = plot_func, legend=legend)
    # plot_lambda_at_one_radius(ax5, "1000Rsun.data", grid_folders=grid_folders1 ,accretor=b1, nonrot=nonrot1, plot_func = plot_func, legend=legend)   
    # # 20 Msun
    bx1 = fig.add_subplot(gs[:20, 50:100])
    bx2 = fig.add_subplot(gs[20:40, 50:100])
    bx3 = fig.add_subplot(gs[40:60, 50:100])
    bx4 = fig.add_subplot(gs[60:80, 50:100])
    bx5 = fig.add_subplot(gs[80:100, 50:100])
    bxes = [bx1, bx2, bx3, bx4, bx5]
    for bx in bxes:
        bx.set_xlim(0, 20.2)
        bx.set_yticklabels([])
        bx.set_ylim(axes[bxes.index(bx)].get_ylim())
        if bx != bxes[-1]:
            bx.set_xticklabels([])
        else:
            bx.set_xlabel(r"$m [M_\odot]$")
    plot_lambda_at_one_radius(bx1, "100Rsun.data", grid_folders=grid_folders2 ,accretor=b2, nonrot=nonrot2, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(bx2, "200Rsun.data", grid_folders=grid_folders2 ,accretor=b2, nonrot=nonrot2, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(bx3, "300Rsun.data", grid_folders=grid_folders2 ,accretor=b2, nonrot=nonrot2, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(bx4, "500Rsun.data", grid_folders=grid_folders2 ,accretor=b2, nonrot=nonrot2, plot_func = plot_func, legend=legend)
    # plot_lambda_at_one_radius(bx5, "1000Rsun.data", grid_folders=grid_folders2 ,accretor=b2, nonrot=nonrot2, plot_func = plot_func, legend=legend)
    # 30
    cx1 = fig.add_subplot(gs[:20, 100:])
    cx2 = fig.add_subplot(gs[20:40, 100:])
    cx3 = fig.add_subplot(gs[40:60, 100:])
    cx4 = fig.add_subplot(gs[60:80, 100:])
    cx5 = fig.add_subplot(gs[80:100, 100:])
    cxes = [cx1, cx2, cx3, cx4, cx5]
    for cx in cxes:
        cx.set_xlim(0, 36.3)
        cx.set_yticklabels([])
        cx.set_ylim(axes[cxes.index(cx)].get_ylim())
        if bx != bxes[-1]:
            cx.set_xticklabels([])
        else:
            cx.set_xlabel(r"$m [M_\odot]$")
    plot_lambda_at_one_radius(cx1, "100Rsun.data", grid_folders=grid_folders3 ,accretor=b3, nonrot=nonrot3, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(cx2, "200Rsun.data", grid_folders=grid_folders3 ,accretor=b3, nonrot=nonrot3, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(cx3, "300Rsun.data", grid_folders=grid_folders3 ,accretor=b3, nonrot=nonrot3, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(cx4, "500Rsun.data", grid_folders=grid_folders3 ,accretor=b3, nonrot=nonrot3, plot_func = plot_func, legend=legend)
    plot_lambda_at_one_radius(cx5, "1000Rsun.data", grid_folders=grid_folders3 ,accretor=b3, nonrot=nonrot3, plot_func = plot_func, legend=legend)
    # ax.set_xlabel(r"m [$M_\odot$]")
    ax3.set_ylabel(r"$\lambda = (GM(M-m)/R)/\mathrm{BE(m)}}$")
    for cx in cxes:
        dx = cx.twinx()
        dx.set_yticks(cx.get_yticks())
        dx.set_yticklabels([])
        if cx == cxes[0]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$100\,R_\odot$"
            X = 100 * Rsun_cm
        if cx == cxes[1]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$200\,R_\odot$"
            X = 200 * Rsun_cm
        if cx == cxes[2]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$300\,R_\odot$"
            X = 300 * Rsun_cm
        if cx == cxes[3]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$500\,R_\odot$"
            X = 500 * Rsun_cm
        if cx == cxes[4]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$1000\,R_\odot$"
            X = 1000 * Rsun_cm
        dx.set_ylabel(label)
    if fig_name:
        plt.savefig(fig_name)

    
grid_lambdas(b1, b2, b3, nonrot18, nonrot20, nonrot36, grid_folders18, grid_folders20, grid_folders36, plot_func=plot_lambda_mass, legend=False)

## surface composition

In [ ]:
for b in [b1,b2,b3]:
    plot_surface_abundances(b+'history.data', legend=True)

## internal structure at onset CE

In [ ]:
def plot_internal_struct(pfile, ax, ls='-'):
    src, col = getSrcCol(pfile)
    logrho = src[:, col.index("logRho")]
    m = src[:, col.index("mass")]
    logr = np.log10(src[:, col.index("radius")]*Rsun_cm)
    h1 = src[:, col.index("h1")]
    he4 = src[:, col.index("he4")]
    c12 = src[:, col.index("c12")]
    ax.plot(logr, logrho, c='c', ls=ls)
    ax.plot(logr, np.log10(h1), c='b', ls=ls)
    ax.plot(logr, np.log10(he4), c='r', ls=ls)
    ax.plot(logr, np.log10(c12), c='m', ls=ls)

In [ ]:
def struct_summary(fig_name=None):
    """
    Parameters:
    ----------
    fig_name: `string` location where to save the figure
    """
    # locations fix with zenodo
    root = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/"
    root_eng = root + "engineered_stars/same_core/"
    # grid_folders18 = sorted(glob.glob(root_eng + "/grid18/*.*/"))
    # grid_folders20 = sorted(glob.glob(root_eng + "/grid20/*.*/"))
    # grid_folders36 = sorted(glob.glob(root_eng + "/grid36/*.*/"))
    ## accretor models
    root_accretors = root + "binaries/Z_0.0019/"
    #
    b1 = (
        root_accretors
        + "/m1_18.0000_m2_15.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/LOGS2/"
    )
    b2 = (
        root_accretors
        + "/m1_20.0000_m2_17.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/LOGS2/"
    )
    b3 = (
        root_accretors
        + "/m1_38.0000_m2_30.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/LOGS2/"
    )

    # single_grid1 = grid_folders18
    # single_grid2 = grid_folders20
    # single_grid3 = grid_folders36

    nonrot18 = root + "single_stars/Z_0.0019/18_rot0.0/"
    nonrot20 = root + "single_stars/Z_0.0019/20_rot0.0/"
    nonrot30 = root + "single_stars/Z_0.0019/36_rot0.0/"

    fig = plt.figure(figsize=(16, 24))
    gs = gridspec.GridSpec(120, 150)
    # accretor15, compare to 18Msun
    ax1 = fig.add_subplot(gs[:20, :50])
    ax2 = fig.add_subplot(gs[20:40, :50])
    ax3 = fig.add_subplot(gs[40:60, :50])
    ax4 = fig.add_subplot(gs[60:80, :50])
    ax5 = fig.add_subplot(gs[80:100, :50])

    axes = [ax1, ax2, ax3, ax4, ax5]
    for ax in axes:
        ax.set_xlim(8.2, 14)
        ax.set_ylim(-11, 4)
        if ax != axes[-1]:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel(r"$\log_{10}(r/\mathrm{[cm]})$")
    
    profiles = sorted(glob.glob(b1 + "/*Rsun.data"))
    for pfile_accretor in profiles:
        if ("10Rsun" in pfile_accretor) or ("20Rsun" in pfile_accretor):
            continue
        string = pfile_accretor.split("/")[-1]
        ax = get_ax_from_pfile(pfile_accretor, axes)
        plot_internal_struct(pfile_accretor, ax, ls='-')
        try:
            pfile_single = glob.glob(nonrot18+'LOGS/'+string)[0]
        except FileNotFoundError:
            continue
        plot_internal_struct(pfile_single, ax, ls='--')
        
    # # # 20 Msun
    bx1 = fig.add_subplot(gs[:20, 50:100])
    bx2 = fig.add_subplot(gs[20:40, 50:100])
    bx3 = fig.add_subplot(gs[40:60, 50:100])
    bx4 = fig.add_subplot(gs[60:80, 50:100])
    bx5 = fig.add_subplot(gs[80:100, 50:100])

    bxes = [bx1, bx2, bx3, bx4, bx5]
    for bx in bxes:
        bx.set_xlim(8.2, 14)
        bx.set_ylim(-11, 4)
        bx.set_yticklabels([])
        if bx != bxes[-1]:
            bx.set_xticklabels([])
        else:
            bx.set_yticklabels([])
            bx.set_xlabel(r"$\log_{10}(r/\mathrm{[cm]})$")

    profiles = sorted(glob.glob(b2 + "/*Rsun.data"))  # , key=get_age_from_profile)
    for pfile_accretor in profiles:
        if ("10Rsun" in pfile_accretor) or ("20Rsun" in pfile_accretor):
            continue
        string = pfile_accretor.split("/")[-1]
        bx = get_ax_from_pfile(pfile_accretor, bxes)
        plot_internal_struct(pfile_accretor, bx, ls='-')
        try:
            pfile_single = glob.glob(nonrot20+'LOGS/'+string)[0]
        except FileNotFoundError:
            continue
        plot_internal_struct(pfile_single, bx, ls='--')
    
    # 30
    cx1 = fig.add_subplot(gs[:20, 100:])
    cx2 = fig.add_subplot(gs[20:40, 100:])
    cx3 = fig.add_subplot(gs[40:60, 100:])
    cx4 = fig.add_subplot(gs[60:80, 100:])
    cx5 = fig.add_subplot(gs[80:100, 100:])

    cxes = [cx1, cx2, cx3, cx4, cx5]
    for cx in cxes:
        cx.set_xlim(8.2, 14)
        cx.set_ylim(-11, 4)
        cx.set_yticklabels([])
        if cx != cxes[-1]:
            cx.set_xticklabels([])
        else:
            cx.set_xlabel(r"$\log_{10}(r/\mathrm{[cm]})$")

    profiles = sorted(glob.glob(b3 + "/*Rsun.data"))  # , key=get_age_from_profile)
    for pfile_accretor in profiles:
        if ("10Rsun" in pfile_accretor) or ("20Rsun" in pfile_accretor):
            continue
        string = pfile_accretor.split("/")[-1]
        cx = get_ax_from_pfile(pfile_accretor, cxes)
        plot_internal_struct(pfile_accretor, cx, ls='-')
        try:
            pfile_single = glob.glob(nonrot30+'LOGS/'+string)[0]
        except FileNotFoundError:
            continue
        plot_internal_struct(pfile_single, cx, ls='--')

    for cx in cxes:
        dx = cx.twinx()
        dx.set_yticks(cx.get_yticks())
        dx.set_yticklabels([])
        if cx == cxes[0]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$100\,R_\odot$"
            X = 100 * Rsun_cm
        if cx == cxes[1]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$200\,R_\odot$"
            X = 200 * Rsun_cm
        if cx == cxes[2]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$300\,R_\odot$"
            X = 300 * Rsun_cm
        if cx == cxes[3]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$500\,R_\odot$"
            X = 500 * Rsun_cm
        if cx == cxes[4]:
            # bx.text(0.2, 0.1, , fontsize=30, transform=bx.transAxes, va="bottom", ha="right")
            label = "$1000\,R_\odot$"
            X = 1000 * Rsun_cm
        dx.set_ylabel(label)
        cx.axvline(np.log10(X), 0, 1, ls="--", lw=2, c="#808080", zorder=0)
        # bx.axhline(0, 0,1, ls='--', lw=2, c="#808080", zorder=0)
        ax = axes[cxes.index(cx)]
        if ax != axes[-1]:  # skip last panel
            ax.axvline(np.log10(X), 0, 1, ls="--", lw=2, c="#808080", zorder=0)
            # ax.axhline(0, 0,1, ls='--', lw=2, c="#808080", zorder=0)
        bx = bxes[cxes.index(cx)]
        if bx != bxes[-1]:  # skip last panel
            bx.axvline(np.log10(X), 0, 1, ls="--", lw=2, c="#808080", zorder=0)

    ax1.set_title(r"$15\rightarrow 18\,M_\odot$", size=30)
    bx1.set_title(r"$17\rightarrow 20\,M_\odot$", size=30)
    cx1.set_title(r"$30\rightarrow 36\,M_\odot$", size=30)

    # legends
    # bxes[-1].plot(np.nan, np.nan, c="k", ls="--", label=r"$\alpha_\mathrm{th}=0$")
    bxes[-1].plot(np.nan, np.nan, c="k", ls="-", label=r"accretor")
    bxes[-1].plot(np.nan, np.nan, c="k", ls="--", label=r"single")
    bxes[-1].legend(loc="center")

    axes[-1].plot(np.nan, np.nan, c="b", ls="-", label=r"$\log_{10}(X(^1\mathrm{H}))$")
    axes[-1].plot(np.nan, np.nan, c="r", ls="-", label=r"$\log_{10}(X(^4\mathrm{He}))$")
    axes[-1].plot(np.nan, np.nan, c="m", ls="-", label=r"$\log_{10}(X(^{12}\mathrm{C}))$")
    axes[-1].plot(np.nan, np.nan, c="c", ls="-", label=r"$\log_{10}(\rho)$")
    axes[-1].legend(handletextpad=0.1, loc="center")
    if fig_name:
        plt.savefig(fig_name)
    
struct_summary(fig_name="../static/summary_internal_structure.pdf")

## find monotonically increasing parameter throughout stellar evolution
to use as indipendent variable for the accretors and single stars

- L, Teff, R don't work because of Henyey hook and blue loops
- M does not work because of accretion
- needs to be some external property
- Tc works!

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:, :])


hfile = nonrot36+'/LOGS/history.data'
src, col = getSrcCol(hfile)
T_center = src[:, col.index("log_center_T")]
L = src[:, col.index("log_L")]
ax.plot(T_center)

hfile = b3+'/history.data'
src, col = getSrcCol(hfile)
T_center = src[:, col.index("log_center_T")]
L = src[:, col.index("log_L")]
ax.plot(T_center)


In [ ]:
# test
fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:, :])
# original
src, col = getSrcCol(nonrot36+'/LOGS/history.data')
env_BE = -1.0*src[:, col.index("envelope_binding_energy")] # MESA assumes alpha_th=1
r = src[:, col.index("radius")]
t = src[:, col.index("star_age")]
T_center = src[:, col.index("log_center_T")]
ax.scatter(T_center, env_BE, s=50)
# interpolated
Tx = np.linspace(min(T_center), max(T_center), 1000)
env_BE_interp = interpolate_BE_env(Tx, T_center, env_BE, offset=1e-50)
ax.scatter(Tx, env_BE_interp, s=10)

In [ ]:
# BE envelope (integrated down the core) as a function of radius
nonrot36 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/single_stars/Z_0.0019/36_rot0.0/"
nonrot20 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/single_stars/Z_0.0019/20_rot0.0/"
nonrot18 = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/single_stars/Z_0.0019/18_rot0.0/"
# src, col = getSrcCol(nonrot30+'LOGS/history.data')
# env_BE_single = -1.0*src[:, col.index("envelope_binding_energy")] # MESA assumes alpha_th=1
# r_single = src[:, col.index("radius")]
# t_single = src[:, col.index("star_age")]

fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:, :])
ax.set_ylabel(r"$BE_\mathrm{env} \mathrm{[erg]}$")
ax.set_xlabel(r"time [yr]")

# sanity check
# ax.plot(t_single, env_BE_single, c='r', label="30Msun")
# xxx = np.linspace(0, max(t_single), len(t_single))
# env_BE_single_interp = interpolate_BE_env(xxx, env_BE_single, t_single)
# ax.plot(xxx, env_BE_single_interp, lw=2, c='k')

for b in [b3]:
    src, col = getSrcCol(b+'history.data')
    env_BE_accretor = -1.0*src[:, col.index("envelope_binding_energy")] # MESA assumes alpha_th=1
    r_accretor = src[:, col.index("radius")]
    t_accretor = src[:, col.index("star_age")]
    T_center_acc = src[:, col.index("log_center_T")]
    # sanity check
    # env_BE_single_interp = interpolate_BE_env(env_BE_accretor, t_accretor*1e-6, env_BE_single, t_single*1e-6)
    ax.plot(T_center_acc, env_BE_accretor, lw=3)

for f in [nonrot36]: # nonrot18, nonrot20, 
    src, col = getSrcCol(f+'LOGS/history.data')
    env_BE = -1.0*src[:, col.index("envelope_binding_energy")] # MESA assumes alpha_th=1
    r = src[:, col.index("radius")]
    t = src[:, col.index("star_age")]
    T_center = src[:, col.index("log_center_T")]
    # sanity check
    # env_BE_single_interp = interpolate_BE_env(env_BE_accretor, t_accretor*1e-6, env_BE_single, t_single*1e-6)
    ax.plot(T_center, env_BE, lw=3, label=f"{f.split('/')[-2].rstrip('_rot0.0')}")
    # interpolate test
    y = interpolate_BE_env(T_center, T_center_acc, env_BE_accretor)
    ax.scatter(T_center, y, label="interp", c='r')
    
ax.set_yscale('log')
ax.legend()

In [ ]:
# plot ratios now
fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:, :])
ax.set_ylabel(r"$BE_\mathrm{env}(\mathrm{accretor})/BE_\mathrm{env}(\mathrm{single})$")
ax.set_xlabel(r"$R\  [R_\odot]$")

for f in [nonrot18, nonrot20, nonrot36]:
    src, col = getSrcCol(f+'LOGS/history.data')
    env_BE = -1.0*src[:, col.index("envelope_binding_energy")] # MESA assumes alpha_th=1
    t = src[:, col.index("star_age")]
    r = src[:, col.index("radius")]
    logTc = src[:, col.index("log_center_T")]
    if "18" in f:
        b = b1
    elif "20" in f:
        b = b2
    elif "36" in f:
        b=b3
    src, col = getSrcCol(b+'history.data')
    # highlight RLOF
    bsrc, bcol = getSrcCol(b+'/../binary_history.data')
    rl_relative_gap_1 = bsrc[:, bcol.index("rl_relative_overflow_1")]
    ind = rl_relative_gap_1 > 0
    env_BE_accretor = -1.0*src[:, col.index("envelope_binding_energy")] # MESA assumes alpha_th=1
    # r_accretor = src[:, col.index("radius")]
    t_accretor = src[:, col.index("star_age")]
    r_accretor = src[:, col.index("radius")]
    logTc_accretor = src[:, col.index("log_center_T")]
    # env_BE_accretor_interp = interpolate_BE_env(t, env_BE_accretor, t_accretor)
    env_BE_accretor_interp = interpolate_BE_env(logTc, logTc_accretor, env_BE_accretor)
    # during RLOF
    # env_BE_accretor_interp_RLOF = interpolate_BE_env(logTc, logTc_accretor[np.logical_not(ind)], env_BE_accretor[np.logical_not(ind)])
    # calculate ratio
    ratio = env_BE_accretor_interp/env_BE
    ax.plot(r, ratio)
    ax.scatter(r[-1], ratio[-1], s=100, marker="*")
    ax.axhline(1,0,1, lw='2', ls='--', c="#808080", zorder=0)
    

In [ ]:
# are BE the same: YES except the sign, and MESA calculates with alpha_th=1
fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:, :])

pfile_single = nonrot36+'LOGS/100Rsun.data'

plot_BE_m(
    pfile_single,
    ax,
    alpha_th=1.0,
    scale_factor=None,
    ls="-",
)
plot_BE_m(
    pfile_single,
    ax,
    alpha_th=0.0,
    scale_factor=None,
    ls="--",
)

bx = ax.twinx()
src, col = getSrcCol(pfile_single)
he = src[:, col.index("he4")]
m = src[:, col.index("mass")]
bx.plot(m, he)


# get value from history file
head, headcol = read_MESA_header(pfile_single)
target_modnum = int(head[headcol.index("model_number")])

hfile = nonrot36+'LOGS/history.data'
src, col  =getSrcCol(hfile)
modnum = src[:, col.index("model_number")]
i = np.argmin(np.absolute(modnum-target_modnum))
ebind = -1.0*src[i, col.index("envelope_binding_energy")]
ax.axhline(ebind, 0,1)
mcore = src[i, col.index("he_core_mass")]
ax.axvline(mcore, 0,1)

